# Practical08_Part1_ModelFreeRL.ipynb

### Import required dependencies

# 1. The Grid World environment

Recall the grid in which our robot lives

![GridWorldExample.png](https://i.postimg.cc/5tMM5vqf/Grid-World-Example.png)

- The states $s \in \mathcal{S}$ correspond to locations in the grid. Each location has also a cell index associated to it, e.g., cell index 4 is associated to location (row=1,col=0)
- The robot can move up, down, left, or right. Actions correpond to unit increments or decrements in the specified direction.
    - Up : (-1,0)
    - Down: (1,0)
    - Left: (0,-1)
    - Right: (0, 1)
- Each action is represented by a number. Action (Up) is represented by 0, (Down) by 1, (Left) by 2 and, finally, (Right) by 3. No actions are available at a terminal state
- Discount factor $\gamma = 0.99$ (class attribute ``gamma=0.99``)
- Stochastic transition matrix (class attribute ``noise=0.2``)
- Rewards are only obtained at terminal states (class attribute ``living_reward=-0.04``)

This environment is represented with the class ``GridEnv``. **To a look at the attributes of this class, place the cursor somewhere on the class' name and hit SHIFT+TAB (local Jupyter Notebook) or hover your mouse over it (Colab). If there's a + button at the top of the popup tooltip, this means the documentation spans a few lines, click it to show the full docstring, then scroll up.**

### Known Model

Recall also the **optimal policy** we found yesterday using policy-interation

![example_policy.png](https://i.postimg.cc/pLjHnkj0/example-policy.png)

since the dynamics of our grid world environment are known, we obtained the state-value function $v_\pi(s)$ associated to this policy using ``policy_evalution(.)`` 

We have defined the class ``GridEnv`` to represent our Grid World MDP.

In [ ]:
# Create a Grid World instance
grid_world = GridEnv(gamma=0.99, noise=0.2, living_reward=-0.04)

# Get policy shown in image
policy_pi = encode_policy(grid_world)

# Compute value-function using dynamic programming
v_pi = policy_evaluation(grid_world, policy_pi)

plot_value_function(grid_world, v_pi)

# 2. Policy Evaluation

## 2.1 Monte-Carlo Method

### What if the Transition and Reward Function are Unknown?

Let's first define the helper method ``generate_episode(.)``. It samples an episode i.e., a sequence of ($s, a, r, s'$) tuples, from a given policy

Now, under the assumption that $\mathcal{T}(s,a,s')$ and $\mathcal{R}(s,a)$ are unknown, let's use the algorithm shown below to get an estimate $\hat{v}_\pi(s)$ of the true state-value function $v_\pi(s)$

![MCPolicyEvaluation.png](https://i.postimg.cc/6pXj5P6D/MCPolicy-Evaluation.png)

Let's now try the algorithm and compare its out to the true value-state function. 

**Interaction**:
Run the algorithm multiple times and observe what happens when the number of episodes increases.

In [ ]:
#change n_episodes to see what happens
errors, predicted_v = monte_carlo_first_visit_policy_evaluation(grid_world, policy_pi, v_pi, n_episodes=100)

fig = plt.figure(constrained_layout=True)
spec = gridspec.GridSpec(ncols=2, nrows=2, figure=fig)
f_ax1 = fig.add_subplot(spec[0, 0])
f_ax2 = fig.add_subplot(spec[0, 1])
f_ax3 = fig.add_subplot(spec[1, :])

#Plot true value function
plot_value_function(grid_world, v_pi, f_ax1)
f_ax1.set_title("True state-value function")

plot_value_function(grid_world, predicted_v, f_ax2)
f_ax2.set_title("Predicted state-value function")

f_ax3.plot(errors)
f_ax3.set_title("Predicted Error (sum of abs. differences)")
f_ax3.set_xlabel("Num. episodes")

# 2.2. Temporal Difference Method
Estimate $\hat{v}_\pi(s)$ of the true state-value function $v_\pi(s)$

![TDPolicyEvaluation.png](https://i.postimg.cc/c4yywX4c/TDPolicy-Evaluation.png)

Let's now try the algorithm and compare its output to the true value-state function. 

In [ ]:
#change n_episodes to see what happens
errors, predicted_v = temporal_learning_policy_evaluation(grid_world, policy_pi, v_pi, alpha=0.1, n_episodes=200)

fig = plt.figure(constrained_layout=True)
spec = gridspec.GridSpec(ncols=2, nrows=2, figure=fig)
f_ax1 = fig.add_subplot(spec[0, 0])
f_ax2 = fig.add_subplot(spec[0, 1])
f_ax3 = fig.add_subplot(spec[1, :])

#Plot true value function
plot_value_function(grid_world, v_pi, f_ax1)
f_ax1.set_title("True state-value function")

plot_value_function(grid_world, predicted_v, f_ax2)
f_ax2.set_title("Predicted state-value function")

f_ax3.plot(errors)
f_ax3.set_title("Predicted Error (sum of abs. differences)")
f_ax3.set_xlabel("Num. episodes")

# 3. Control

In a model-free setting where our state-value and action-value estimates depend on the actions chosen by the agent, how can we guarantee that the all actions will continue to be selected?

## 3.1 $\epsilon$-Greedy Policies

We can use an $\epsilon$-greedy policy. This type of policy are formally defined as:

\begin{equation*}
\begin{aligned}
    \pi(a|s) = 
    \begin{cases}
        1 - \epsilon + \frac{\epsilon}{|\mathcal{A}|},&  \text{if } a^* = \arg\max_{a \in \mathcal{A}} q_\pi(s,a)\\
        \frac{\epsilon}{|\mathcal{A}|}, & \text{otherwise}
    \end{cases}
\end{aligned}
\end{equation*}

Let's see how the agent behaves when it follows an $\epsilon$-greedy policy.

In [ ]:
# We set noise to zero. Randomness in agent behaviour is only due to e-greedy policy
grid_world = GridEnv(noise=0, living_reward=-0.04, gamma=0.99)

# Get policy shown in section 1
policy_pi = encode_policy(grid_world)

# Compute value-function using dynamic programming
v_pi = policy_evaluation(grid_world, policy_pi)

# Use value-function to compute q-values
q_pi = grid_world.get_q_values(v_pi)

# Start episode
cur_state = grid_world.idx_cur_state
s_x, s_y = get_state_to_plot(grid_world)

# We can visualize our grid world using the render() function
fig, ax = grid_world.render()
agent, = ax.plot([], [], 'o', color='b', linewidth=6)
reward_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)

done = False
cumulative_reward = 0
path_to_plot = []

v_epsilon = 0.8

while not done:
    action = get_egreedy_action(grid_world, cur_state, q_pi, v_epsilon)
    cur_state, cur_reward, done, _ = grid_world.step(int(action))
    n_x, n_y = get_state_to_plot(grid_world)
    cumulative_reward += cur_reward
    path_to_plot.append([cumulative_reward, n_x, n_y])

def init():
    agent.set_data(s_x + 0.5, s_y + 0.5)
    reward_text.set_text('')
    return agent, reward_text

def animate(i):
    if i < len(path_to_plot):
        r, n_x, n_y = path_to_plot[i]
        agent.set_data(n_x + 0.5, n_y + 0.5)
        reward_text.set_text('Cumulative reward: %.2f' % r)
    return agent, reward_text

ani = animation.FuncAnimation(fig, animate, frames=len(path_to_plot), blit=False, interval=500, init_func=init,
                              repeat=False)

plt.close('all') 
display(HTML(f"<div align=\"center\">{ani.to_jshtml()}</div>"))

## 3.2 Q-Learning

We have seen how to evaluate a policy without a model. Let's now find an *approximately* optimal policy using the off-policy control method Q-learning.

To help during the learning, we have added a lambda function that iteratively decreases epsilon. Our agent will strongly explore the environment at first to then swicth into exploitation mode

In [ ]:
fig, ax = plt.subplots(figsize=(4, 4))

ax.plot([epsilon_by_episode(i) for i in range(500)])
ax.set_xlabel("Num. episodes")
ax.set_ylabel("Epsilon")

Here is our implementation of the q-learning algorithm shown below

![q-learning.png](https://i.postimg.cc/8z70Yv5C/q-learning.png)

**Complete the missing steps**:
- Choose an action using an $\epsilon$-greedy policy (use the function ``get_egreedy_action(.)`` we tested in Section 3.1)
- Update our q-function using a greedy (max) policy (use ``q_function[cur_state][action]`` to index our q-function)

**Keep in Mind**: Correspondance between the mathematical notation and implemented code

|                 |                            |                  |
| :-------------- | -------------------------: | ---------------: |
|                 | **Variable/Attribute**     | **Type**         | 
| $\epsilon$      | `epsilon_by_episode`       | `float`          |
| $\alpha$        | `alpha`                    | `float`          | 
| $\gamma$        | `grid_world.gamma`         | `float`          | 
| $\hat{q}(s, a)$ | `q_function[idx_s][idx_a]` | `dict` of `dict` | 
| $s$             | `cur_state`                | `int`            | 
| $r$             | `reward`                   | `int`            |
| $s^{\prime}$    | `next_state`               | `int`            |

Let's now test our implementation and compare our free-model policy with the one we obtained in the last lecture using value iteration

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d30c9d78-b335-4e3c-b9dd-5dae33556494' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>